In [ ]:
from GEOparse import *
import pandas as pd
import numpy as np
import os
import sys

In [10]:
# retrieve all strain phenotypes available
strains_genotyped = pd.DataFrame(pd.read_table("./data/jbloom_strains_genotyped.tab"))
# find a place where the metadata starts to dump it
last_segregant_col = strains_genotyped.columns.tolist().index("17_1_a")


# sort the strain names
def strain_to_list(item):
    l = item.split('_')
    return [int(l[0]), int(l[1]), ord(l[2])]

strain_names_full = sorted(
    strains_genotyped.columns.tolist()[1:last_segregant_col+1],
    key=strain_to_list
)

# Display all genotyped strains 
print("{}: {}".format(
        len(strain_names_full), 
        strain_names_full
    )
)

124: ['1_1_d', '1_2_d', '1_3_d', '1_4_d', '1_5_c', '2_1_d', '2_2_d', '2_3_d', '2_4_a', '2_5_d', '2_6_d', '2_7_a', '2_7_b', '2_7_c', '2_7_d', '3_1_d', '3_2_d', '3_3_d', '3_4_d', '3_5_d', '4_1_c', '4_2_a', '4_3_d', '4_4_d', '5_1_d', '5_2_d', '5_3_d', '5_4_d', '5_5_d', '6_1_d', '6_2_b', '6_3_c', '6_4_d', '6_5_d', '6_6_d', '6_7_d', '7_1_d', '7_2_c', '7_3_d', '7_4_c', '7_5_d', '7_6_c', '7_7_c', '7_8_d', '8_1_a', '8_2_d', '8_3_a', '8_4_c', '8_5_b', '8_6_c', '8_7_b', '9_1_d', '9_2_d', '9_3_d', '9_4_d', '9_5_d', '9_6_d', '9_7_d', '10_1_c', '10_2_d', '10_3_c', '10_4_d', '11_1_a', '11_2_d', '11_3_b', '12_1_d', '12_2_b', '13_1_a', '13_2_c', '13_3_b', '13_4_a', '13_5_c', '14_1_b', '14_2_c', '14_3_d', '14_4_a', '14_5_b', '14_6_d', '14_7_c', '15_2_d', '15_3_b', '15_4_d', '15_5_b', '15_6_c', '16_1_d', '17_1_a', '17_2_d', '17_4_a', '17_5_b', '18_1_d', '18_2_d', '18_3_d', '18_4_c', '18_6_d', '19_1_c', '19_2_c', '19_3_c', '19_4_b', '19_5_b', '20_1_d', '20_2_d', '20_3_c', '20_4_c', '20_5_d', '21_1_d', '2

In [10]:
# Parse GDS with experiment data
# Ignore annoying "Unknown subset type" warnings
null_device = open(os.devnull, 'w')
original_stderr = sys.__stderr__
sys.__stderr__ = null_device  

GDS1115 = GEOparse.parse_GDS(
    filepath="./data/GDS1115.soft"
)
   
GDS1116 = GEOparse.parse_GDS(
    filepath="./data/GDS1116.soft"
)
# I wasn't provided with genotype data for 23_2_d,
# hence, I dropped the corresponding columns
GDS1115.table.drop("GSM62170", axis=1, inplace=True)
GDS1116.table.drop("GSM62171", axis=1, inplace=True)
sys.__stderr__ = original_stderr

 - DATABASE : Geo
 - DATASET : GDS1115
 - SUBSET : GDS1115_1
 - SUBSET : GDS1115_2
 - SUBSET : GDS1115_3
 - DATASET : GDS1115
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GD

 - DATABASE : Geo
 - DATASET : GDS1116
 - SUBSET : GDS1116_1
 - SUBSET : GDS1116_2
 - SUBSET : GDS1116_3
 - DATASET : GDS1116
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_2
Unknown subset type: strain for subset GDS1116_1
Unknown subset type: strain for subset GDS1116_1
Unknown subset type: strain for subset GDS1116_1
Unknown subset type: strain for subset GDS1116_1
Unknown subset type: strain for subset GDS1116_1
Unknown subset type: strain for subset GD

In [13]:
# Read the strain ids used in the paper
strain_data_brem = pd.read_csv('./data/brem2005_strains.csv')
strain_names_brem = strain_data_brem['Strain name'].tolist()
# Extract expression data from two microarrays
# into numpy matrices
GDS1115_m = pd.DataFrame.copy(GDS1115.table.iloc[:, 2:])\
                        .convert_objects(convert_numeric=True).values
GDS1116_m = pd.DataFrame.copy(GDS1116.table.iloc[:, 2:])\
                        .convert_objects(convert_numeric=True).values
expr_avg_m = np.zeros(shape=GDS1115_m.shape)
# Extract averaged expression data:
# -- If both values are available, take their mean
# -- If only one value is available, take that value
# -- Otherwise, set expression value to numpy.nan
for i in range(GDS1115_m.shape[0]):
    for j in range(GDS1115_m.shape[1]):
        if not np.isnan(GDS1115_m[i, j]) and not np.isnan(GDS1116_m[i, j]):
            expr_avg_m[i, j] = (GDS1115_m[i, j] + GDS1116_m[i, j]) / 2.
        elif not np.isnan(GDS1115_m[i, j]):
            expr_avg_m[i, j] = GDS1115_m[i, j]
        elif not np.isnan(GDS1116_m[i, j]):
            expr_avg_m[i, j] = GDS1116_m[i, j]
        else:
            expr_avg_m[i, j] = np.nan
# Construct resulting dataframe with expression data,g
# averaged and cleaned up. Firstly, replace irrelevant
# GSMs with strain names
expr_df = pd.DataFrame(
    data=expr_avg_m,
    columns=strain_names_brem
)

/home/vvi/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:6: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


/home/vvi/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:7: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  import sys


In [1]:
# Then, insert columns "ID_REF" and "IDENTIFIER"
# to make new dataframe similar to old ones
expr_df.insert(loc=0, column="ID_REF", value=0)
expr_df.insert(loc=1, column="IDENTIFIER", value='')
# And fill those columns with annotation values
expr_df["ID_REF"] = GDS1115.table["ID_REF"]
expr_df["IDENTIFIER"] = GDS1115.table["IDENTIFIER"]
# Drop the empty rows
expr_df = expr_df[(expr_df.IDENTIFIER != 'blank') 
        & (expr_df.IDENTIFIER != 'empty')]
# And save everything to csv file
expr_df.to_csv(
    './data/brem2005_RNA_expression.csv',
    index=False,
    sep='\t'
)

NameError: name 'expr_df' is not defined

In [2]:
# BY and RM strains values must be averaged and placed into two separate columns

expression_df = pd.read_table('./data/brem2005_RNA_expression.csv')
genotypes_df_raw = pd.read_table('./data/jbloom_strains_genotyped.tab')

# Add columns with the parental genotypes
# to use their averaged expression data

genotypes_df_raw.insert(
    loc=1, 
    column="BY", 
    value=np.zeros(markers_n, dtype="int")
) 
genotypes_df_raw.insert(
    loc=2,
    column="RM",
    value=np.ones(markers_n, dtype="int")
)

NameError: name 'pd' is not defined

In [ ]:
# Average the expression values over all parental strains.
# Add averaged columns called "BY" and "RM" to expression dataframe
# and remove the parental strains' columns

BY_expr_df = expression_df.iloc[:, 2:8]
RM_expr_df = expression_df.iloc[:, 8:20]


def avg_expr_df(df):
    row_n, col_n = df.shape 
    averaged = np.full(shape=row_n, fill_value=np.nan)
    # iterate over rows of a dataframe
    for i, row in enumerate(df.itertuples()):
        expr_sum, cnt = 0., 0.
        # and average non-NaN values of the columns
        # presence of NaNs is exactly the reason why the columns 
        # cannot be simply added together and averaged
        for j in range(1, col_n + 1):
            if not np.isnan(row[j]):
                expr_sum += row[j]
                cnt += 1
        if cnt:
            averaged[i] = expr_sum / cnt
    return averaged


pd.DataFrame.to_csv(
    expression_df,
    "./data/rna_expression_full.csv",
    sep='\t'
)

expression_df.insert(
    loc=2,
    column="BY",
    value=pd.DataFrame(avg_expr_df(BY_expr_df))
)

expression_df.insert(
    loc=3,
    column="RM",
    value=pd.DataFrame(avg_expr_df(RM_expr_df))
)

# Non-averaged expression data of parental strains is no longer necessary,
# so the appropriate columns are dropped from the dataframe


expression_df = expression_df.drop(
    BY_expr_df.columns.tolist() + RM_expr_df.columns.tolist(), 
    axis=1
)

pd.DataFrame.to_csv(
    expression_df,
    "./data/rna_expression_avg.csv",
    sep='\t'
)

In [ ]:
strain_names = expression_df.columns.tolist()[2:]
strain_names

pd.DataFrame.to_csv(
    genotypes_df_raw[["RQTL_name"] + strain_names],
    "./data/genotypes.csv",
    sep='\t'
)

pd.DataFrame.to_csv(
    genotypes_df_raw[
        ["RQTL_name", "name", "chromosome", 
         "position", "sequence", 
         "rec_fractions", "_affyID"]
    ],
    "./data/genotypes_annotation.csv",
    sep='\t'
)
